<a href="https://colab.research.google.com/github/FukuokaIruka/Ex3/blob/main/jikken3_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gensim
!pip install mecab-python3
!pip install unidic-lite

gensim 戦争してた
mecab なんかいる
unidic-lite あのー日本語のやつ　　らしいぞ
エラーが出たらランタイム再起動セッション

下の奴はやるなと。もう遅い!bitch


In [ ]:
# !mkdir data
# !wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip -P data/
# !unzip data/ja.text8.zip -d data/

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec, Text8Corpus
import numpy as np

sample_sents = [['you', 'say', 'goodbye', 'and', 'I', "say", "hello" '.']]
model = Word2Vec(sentences=sample_sents, vector_size=5, window=1, min_count=1)

作ったやつがどうたら。。知らん

window is how close to that shit. for each closure??? IDK
Having absolutely no idea is crazy

In [ ]:
model.wv['you']

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = Text8Corpus('data/ja.text8')
model = Word2Vec(sentences, vector_size=100, window=5, sg=1)

the thingie above costs 10 minutes but he said it after i clicked so it's not my responsibility tbh lmaoo die fast cpu

mine done faster than teacher shii

In [ ]:
import MeCab

def tokenize(text):
    """ テキストを形態素解析して、トークンのリストを返す """
    mecab = MeCab.Tagger("-Owakati -rdictionary/")
    return mecab.parse(text).strip().split()

I don't understand quite tbh. The thingie doing training still. Hopefully all I have to do is wait.

In [ ]:
!mkdir dictionary
#!cat /dev/null > ./dicrc

In [ ]:
documents = ["これはサンプルの文書です。", "Word2Vecの学習を行います。"]

# 形態素解析を行い、単語リストに変換
tokenized_documents = [tokenize(doc) for doc in documents]

In [ ]:
tokenized_documents

In [ ]:
!head --bytes 300 data/ja.text8

よく見ればもうスペースで別れている。びっくりだよ

---
それにしても下のコードの実行が遅すぎる。しね


In [ ]:
model = Word2Vec(
    sentences=sentences,  # 学習対象のコーパス
    vector_size=100,             # 埋め込みベクトルの次元数
    window=5,                    # コンテキストウィンドウのサイズ
    min_count=5,                 # 登場頻度が5未満の単語は無視
    sg=1,                        # 1でskip-gramを使用（0ならCBOW）
    negative=5                   # negative samplingのサンプル数
)

In [ ]:
model.wv['日本']

いろいろなたんごがどうたら。どれぐらい似ているかを計算できるとか。これもNormalisationの犠牲なんだろうか。漂白だあ

$similarity(w_1, w_2) = \frac{w_1 \cdot w_2}{|w_1||w_2|}$

In [ ]:
model.wv.similarity('猫', '犬')

In [ ]:
model.wv.most_similar('国王', topn=10)

In [ ]:
maybe_queen = model.wv['王'] - model.wv['男'] + model.wv['女']
maybe_queen

In [ ]:
model.wv.most_similar(maybe_queen, topn=10)

In [ ]:
np.dot(maybe_queen, model.wv['王妃']) / (np.linalg.norm(maybe_queen)) * np.linalg.norm(model.wv['王妃'])

In [ ]:
model.wv.most_similar('男', topn=10)

In [ ]:
maybe_tokyo = model.wv['ロンドン'] - model.wv['イギリス'] + model.wv['日本']
model.wv.most_similar(maybe_tokyo, topn=10)

。ソチて。その切り方はなくないか

In [ ]:
what_animal = (model.wv['犬'] + model.wv['猫']) / 2
model.wv.most_similar(what_animal, topn=10)

In [ ]:
import copy

all_words = copy.copy(model.wv.index_to_key)
len(all_words)

辞書には75kの単語が。

In [ ]:
all_words[600:700]
# words

In [ ]:
from sklearn.manifold import TSNE

# words = all_words[34600:35000]
# # words = ['教授', '先生', '学生', '兵士', '犬', '猫', '狐', 'ウサギ']
# vector_words = model.wv[words]
# tsne = TSNE(n_components=2, random_state=42, perplexity=3)
# vectors_tsne = tsne.fit_transform(vector_words)

vector_words = model.wv[words]
tsne = TSNE(n_components=2, random_state=42, perplexity=3)
vectors_tsne = tsne.fit_transform(vector_words)

t-sne thingie can be done atp. High dementional thingie slapped onto 2 dimentional map.

In [ ]:
!pip install japanize-matplotlib

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.figure(figsize=(10, 10))
for i, word in enumerate(words):
    plt.scatter(vectors_tsne[i, 0], vectors_tsne[i, 1])
    plt.annotate(word, (vectors_tsne[i, 0], vectors_tsne[i, 1]))

In [ ]:
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
# XというデータをK-Means法によって分析する
y_pred = kmeans.fit_predict(vector_words)

In [ ]:
import pandas as pd

words_frame = pd.DataFrame(vector_words)
words_frame['label'] = kmeans.labels_

In [ ]:
plt.figure(figsize=(10, 10))
for i, word in enumerate(words):
    plt.annotate(word, (vectors_tsne[i, 0], vectors_tsne[i, 1]))
plt.scatter(vectors_tsne[:,0], vectors_tsne[:,1], c=words_frame['label'], cmap='Dark2')

まあよくよく考えれば。。。1000ベクトルぐらいあるやつを圧縮してマップにしてあるわけだから精度がさがるのはしょうがないみたいな。らしいぞ。

In [ ]:
model.wv.similarity('ペロ', 'けど')

Ikyaneseではすげえmake senseだが、日本語由来ではなくスペイン語由来なので、そう高い数字は出ていない。うん。

In [ ]:
what_but = (model.wv['ペロ'] + model.wv['けど']) / 2
model.wv.most_similar(what_but, topn=10)

絶望的に"違う"んだよな

In [ ]:
what_hey = (model.wv['ねえ'] + model.wv['お']) / 2
model.wv.most_similar(what_hey, topn=10)

In Ikyanese we greets with "o", and clearly no AI can understand Ikanese just by japanese knowledge. Human can, tho.